In [10]:
# A-1 TF-IDF文本相似度計算
# ==================== 測試資料 ====================

documents = [
    "人工智慧正在改變世界，機器學習是其核心技術",
    "深度學習推動了人工智慧的發展，特別是在圖像識別領域",
    "今天天氣很好，適合出去運動",
    "機器學習和深度學習都是人工智慧的重要分支",
    "運動有益健康，每天都應該保持運動習慣"
]

In [11]:
import math
from collections import Counter

# ==================== 第一部分：手動計算 TF-IDF ====================

def calculate_tf(word_dict, total_words):
    """
    計算詞頻
    Args:
        word_dict: 詞彙計數字典
        total_words: 總詞數
    Returns:
        tf_dict: TF 值字典
    """
    tf_dict = {}
    for word, count in word_dict.items():
        tf_dict[word] = count / total_words
    return tf_dict


def calculate_idf(documents, word):
    """
    計算逆文件頻率
    Args:
        documents: 文件列表
        word: 目標詞彙
    Returns:
        idf: IDF 值
    """
    # 計算包含該詞的文件數
    num_docs_containing_word = sum(1 for doc in documents if word in doc)

    # 如果詞不存在於任何文件中，返回0
    if num_docs_containing_word == 0:
        return 0

    # IDF = log(總文件數 / 包含該詞的文件數)
    idf = math.log(len(documents) / num_docs_containing_word)
    return idf


def calculate_tfidf_manual(documents):
    """
    手動計算所有文件的 TF-IDF
    """
    tfidf_vectors = []

    # 建立所有詞彙的集合
    all_words = set()
    for doc in documents:
        all_words.update(doc.split())

    # 對每個文件計算 TF-IDF
    for doc in documents:
        words = doc.split()
        word_count = Counter(words)
        total_words = len(words)

        # 計算 TF
        tf_dict = calculate_tf(word_count, total_words)

        # 計算 TF-IDF
        tfidf_dict = {}
        for word in all_words:
            if word in tf_dict:
                tf = tf_dict[word]
                idf = calculate_idf(documents, word)
                tfidf_dict[word] = tf * idf
            else:
                tfidf_dict[word] = 0

        tfidf_vectors.append(tfidf_dict)

    return tfidf_vectors, sorted(all_words)


def cosine_similarity_manual(vec1, vec2, all_words):
    """
    手動計算餘弦相似度
    """
    dot_product = sum(vec1[word] * vec2[word] for word in all_words)

    magnitude1 = math.sqrt(sum(vec1[word] ** 2 for word in all_words))
    magnitude2 = math.sqrt(sum(vec2[word] ** 2 for word in all_words))

    if magnitude1 == 0 or magnitude2 == 0:
        return 0

    return dot_product / (magnitude1 * magnitude2)

In [12]:
# 手動計算
tfidf_vectors_manual, all_words = calculate_tfidf_manual(documents)

print(f"\n詞彙表 ({len(all_words)} 個詞):")
print(all_words[:10], "...")  # 只顯示前10個詞

print("\n各文件的 TF-IDF 向量 (部分顯示):")
for i, vec in enumerate(tfidf_vectors_manual):
    print(f"\n文件 {i+1}: {documents[i][:30]}...")
    # 只顯示非零的 TF-IDF 值
    non_zero = {k: f"{v:.4f}" for k, v in vec.items() if v > 0}
    print(f"  非零 TF-IDF 值數量: {len(non_zero)}")
    print(f"  前3個值: {dict(list(non_zero.items())[:3])}")

print("\n" + "=" * 80)
print("文件相似度矩陣 (手動計算):")
print("=" * 80)

# 計算所有文件對的相似度
similarity_matrix_manual = []
for i in range(len(documents)):
    row = []
    for j in range(len(documents)):
        sim = cosine_similarity_manual(tfidf_vectors_manual[i],
                                      tfidf_vectors_manual[j],
                                      all_words)
        row.append(sim)
    similarity_matrix_manual.append(row)

# 顯示相似度矩陣
print("\n     ", end="")
for i in range(len(documents)):
    print(f"文件{i+1:2d}  ", end="")
print()

for i, row in enumerate(similarity_matrix_manual):
    print(f"文件{i+1:2d} ", end="")
    for val in row:
        print(f"{val:7.4f} ", end="")
    print()

print("\n文件對相似度分析:")
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        sim = similarity_matrix_manual[i][j]
        print(f"文件{i+1} vs 文件{j+1}: {sim:.4f}")
        print(f"  文件{i+1}: {documents[i][:40]}...")
        print(f"  文件{j+1}: {documents[j][:40]}...")
        print()


詞彙表 (5 個詞):
['人工智慧正在改變世界，機器學習是其核心技術', '今天天氣很好，適合出去運動', '機器學習和深度學習都是人工智慧的重要分支', '深度學習推動了人工智慧的發展，特別是在圖像識別領域', '運動有益健康，每天都應該保持運動習慣'] ...

各文件的 TF-IDF 向量 (部分顯示):

文件 1: 人工智慧正在改變世界，機器學習是其核心技術...
  非零 TF-IDF 值數量: 1
  前3個值: {'人工智慧正在改變世界，機器學習是其核心技術': '1.6094'}

文件 2: 深度學習推動了人工智慧的發展，特別是在圖像識別領域...
  非零 TF-IDF 值數量: 1
  前3個值: {'深度學習推動了人工智慧的發展，特別是在圖像識別領域': '1.6094'}

文件 3: 今天天氣很好，適合出去運動...
  非零 TF-IDF 值數量: 1
  前3個值: {'今天天氣很好，適合出去運動': '1.6094'}

文件 4: 機器學習和深度學習都是人工智慧的重要分支...
  非零 TF-IDF 值數量: 1
  前3個值: {'機器學習和深度學習都是人工智慧的重要分支': '1.6094'}

文件 5: 運動有益健康，每天都應該保持運動習慣...
  非零 TF-IDF 值數量: 1
  前3個值: {'運動有益健康，每天都應該保持運動習慣': '1.6094'}

文件相似度矩陣 (手動計算):

     文件 1  文件 2  文件 3  文件 4  文件 5  
文件 1  1.0000  0.0000  0.0000  0.0000  0.0000 
文件 2  0.0000  1.0000  0.0000  0.0000  0.0000 
文件 3  0.0000  0.0000  1.0000  0.0000  0.0000 
文件 4  0.0000  0.0000  0.0000  1.0000  0.0000 
文件 5  0.0000  0.0000  0.0000  0.0000  1.0000 

文件對相似度分析:
文件1 vs 文件2: 0.0000
  文件1: 人工智慧正在改變世界，機器學習是其核心技術...
  文件2: 深度學習推動了人工智慧的發展，特別是在圖像識別領域...


In [13]:
# ==================== 第二部分：使用 scikit-learn 實作 ====================

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def calculate_tfidf_sklearn(documents):
    """
    使用 scikit-learn 計算 TF-IDF 和相似度
    """
    # 建立 TF-IDF 向量化器
    vectorizer = TfidfVectorizer()

    # 計算 TF-IDF 矩陣
    tfidf_matrix = vectorizer.fit_transform(documents)

    # 計算餘弦相似度矩陣
    similarity_matrix = cosine_similarity(tfidf_matrix)

    return tfidf_matrix, similarity_matrix, vectorizer

In [14]:
# 使用 scikit-learn 計算
tfidf_matrix, similarity_matrix_sklearn, vectorizer = calculate_tfidf_sklearn(documents)

print(f"\nTF-IDF 矩陣形狀: {tfidf_matrix.shape}")
print(f"詞彙表大小: {len(vectorizer.get_feature_names_out())}")
print(f"詞彙表 (前10個): {vectorizer.get_feature_names_out()[:10]}")

print("\n" + "=" * 80)
print("文件相似度矩陣 (scikit-learn):")
print("=" * 80)

print("\n     ", end="")
for i in range(len(documents)):
    print(f"文件{i+1:2d}  ", end="")
print()

for i, row in enumerate(similarity_matrix_sklearn):
    print(f"文件{i+1:2d} ", end="")
    for val in row:
        print(f"{val:7.4f} ", end="")
    print()

print("\n文件對相似度分析:")
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        sim = similarity_matrix_sklearn[i][j]
        print(f"文件{i+1} vs 文件{j+1}: {sim:.4f}")
        print(f"  文件{i+1}: {documents[i][:40]}...")
        print(f"  文件{j+1}: {documents[j][:40]}...")
        print()


print("\n" + "=" * 80)
print("結論")
print("=" * 80)

print("\n相似度最高的文件對:")
max_sim = 0
max_pair = (0, 0)
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        if similarity_matrix_sklearn[i][j] > max_sim:
            max_sim = similarity_matrix_sklearn[i][j]
            max_pair = (i, j)

i, j = max_pair
print(f"文件{i+1} 和 文件{j+1}, 相似度: {max_sim:.4f}")
print(f"  文件{i+1}: {documents[i]}")
print(f"  文件{j+1}: {documents[j]}")

print("\n相似度最低的文件對:")
min_sim = 1
min_pair = (0, 0)
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        if similarity_matrix_sklearn[i][j] < min_sim:
            min_sim = similarity_matrix_sklearn[i][j]
            min_pair = (i, j)

i, j = min_pair
print(f"文件{i+1} 和 文件{j+1}, 相似度: {min_sim:.4f}")
print(f"  文件{i+1}: {documents[i]}")
print(f"  文件{j+1}: {documents[j]}")


TF-IDF 矩陣形狀: (5, 9)
詞彙表大小: 9
詞彙表 (前10個): ['人工智慧正在改變世界' '今天天氣很好' '機器學習和深度學習都是人工智慧的重要分支' '機器學習是其核心技術' '每天都應該保持運動習慣'
 '深度學習推動了人工智慧的發展' '特別是在圖像識別領域' '運動有益健康' '適合出去運動']

文件相似度矩陣 (scikit-learn):

     文件 1  文件 2  文件 3  文件 4  文件 5  
文件 1  1.0000  0.0000  0.0000  0.0000  0.0000 
文件 2  0.0000  1.0000  0.0000  0.0000  0.0000 
文件 3  0.0000  0.0000  1.0000  0.0000  0.0000 
文件 4  0.0000  0.0000  0.0000  1.0000  0.0000 
文件 5  0.0000  0.0000  0.0000  0.0000  1.0000 

文件對相似度分析:
文件1 vs 文件2: 0.0000
  文件1: 人工智慧正在改變世界，機器學習是其核心技術...
  文件2: 深度學習推動了人工智慧的發展，特別是在圖像識別領域...

文件1 vs 文件3: 0.0000
  文件1: 人工智慧正在改變世界，機器學習是其核心技術...
  文件3: 今天天氣很好，適合出去運動...

文件1 vs 文件4: 0.0000
  文件1: 人工智慧正在改變世界，機器學習是其核心技術...
  文件4: 機器學習和深度學習都是人工智慧的重要分支...

文件1 vs 文件5: 0.0000
  文件1: 人工智慧正在改變世界，機器學習是其核心技術...
  文件5: 運動有益健康，每天都應該保持運動習慣...

文件2 vs 文件3: 0.0000
  文件2: 深度學習推動了人工智慧的發展，特別是在圖像識別領域...
  文件3: 今天天氣很好，適合出去運動...

文件2 vs 文件4: 0.0000
  文件2: 深度學習推動了人工智慧的發展，特別是在圖像識別領域...
  文件4: 機器學習和深度學習都是人工智慧的重要分支...

文件2 vs 文件5: 0.0000
  文件2: 深度學習推動了人工智慧

In [15]:
#A-2 基於規則的文本分類
# ==================== 測試資料 ====================

test_texts = [
    "這家餐廳的牛肉麵真的太好吃了，湯頭濃郁，麵條Q彈，下次一定再來！",
    "最新的AI技術突破讓人驚艷，深度學習模型的表現越來越好",
    "這部電影劇情空洞，演技糟糕，完全是浪費時間",
    "每天慢跑5公里，配合適當的重訓，體能進步很多"
]

In [16]:
# ==================== 1. 情感分類器  ====================

class RuleBasedSentimentClassifier:
    def __init__(self):
        # 建立正負面詞彙庫
        self.positive_words = ['好', '棒', '優秀', '喜歡', '推薦',
                               '滿意', '開心', '值得', '精彩', '完美']

        self.negative_words = ['差', '糟', '失望', '討厭', '不推薦',
                               '浪費', '無聊', '爛', '糟糕', '差勁']

        # 加入否定詞處理
        self.negation_words = ['不', '沒', '無', '非', '別']

    def classify(self, text):
        """
        分類邏輯:
        1. 計算正負詞數量
        2. 處理否定詞 (否定詞+正面詞=負面)
        3. 考慮程度副詞的加權
        4. 返回: 正面/負面/中性
        """
        positive_count = 0
        negative_count = 0

        # 程度副詞加權
        degree_words = {
            '很': 1.5,
            '非常': 2.0,
            '超': 2.0,
            '太': 1.8,
            '特別': 1.5,
            '真的': 1.3,
            '真': 1.3,
            '十分': 1.8,
            '相當': 1.5
        }

        # 將文本轉換為字符列表進行分析
        words = list(text)

        # 遍歷文本
        i = 0
        while i < len(text):
            # 檢查是否有程度副詞
            degree_multiplier = 1.0
            for degree_word, weight in degree_words.items():
                if text[i:i+len(degree_word)] == degree_word:
                    degree_multiplier = weight
                    i += len(degree_word)
                    break

            # 檢查是否有否定詞
            is_negated = False
            for neg_word in self.negation_words:
                if text[i:i+len(neg_word)] == neg_word:
                    is_negated = True
                    i += len(neg_word)
                    break

            # 檢查正面詞
            found = False
            for pos_word in self.positive_words:
                if text[i:i+len(pos_word)] == pos_word:
                    if is_negated:
                        # 否定詞 + 正面詞 = 負面
                        negative_count += degree_multiplier
                    else:
                        positive_count += degree_multiplier
                    i += len(pos_word)
                    found = True
                    break

            if found:
                continue

            # 檢查負面詞
            for neg_word in self.negative_words:
                if text[i:i+len(neg_word)] == neg_word:
                    if is_negated:
                        # 否定詞 + 負面詞 = 正面
                        positive_count += degree_multiplier
                    else:
                        negative_count += degree_multiplier
                    i += len(neg_word)
                    found = True
                    break

            if not found:
                i += 1

        # 判斷情感
        if positive_count > negative_count:
            sentiment = '正面'
            score = positive_count - negative_count
        elif negative_count > positive_count:
            sentiment = '負面'
            score = negative_count - positive_count
        else:
            sentiment = '中性'
            score = 0

        return {
            'sentiment': sentiment,
            'positive_score': positive_count,
            'negative_score': negative_count,
            'confidence': score
        }


# ==================== 情感分類器測試 ====================

print("=" * 80)
print("1. 情感分類器測試")
print("=" * 80)

sentiment_classifier = RuleBasedSentimentClassifier()

for i, text in enumerate(test_texts, 1):
    print(f"\n測試文本 {i}: {text}")
    result = sentiment_classifier.classify(text)
    print(f"  情感: {result['sentiment']}")
    print(f"  正面分數: {result['positive_score']:.2f}")
    print(f"  負面分數: {result['negative_score']:.2f}")
    print(f"  信心度: {result['confidence']:.2f}")

print("\n" + "=" * 80)
print("情感分類統計")
print("=" * 80)

sentiment_stats = {'正面': 0, '負面': 0, '中性': 0}
for text in test_texts:
    result = sentiment_classifier.classify(text)
    sentiment_stats[result['sentiment']] += 1

print(f"正面文本數量: {sentiment_stats['正面']}")
print(f"負面文本數量: {sentiment_stats['負面']}")
print(f"中性文本數量: {sentiment_stats['中性']}")


1. 情感分類器測試

測試文本 1: 這家餐廳的牛肉麵真的太好吃了，湯頭濃郁，麵條Q彈，下次一定再來！
  情感: 正面
  正面分數: 1.00
  負面分數: 0.00
  信心度: 1.00

測試文本 2: 最新的AI技術突破讓人驚艷，深度學習模型的表現越來越好
  情感: 正面
  正面分數: 1.00
  負面分數: 0.00
  信心度: 1.00

測試文本 3: 這部電影劇情空洞，演技糟糕，完全是浪費時間
  情感: 負面
  正面分數: 0.00
  負面分數: 2.00
  信心度: 2.00

測試文本 4: 每天慢跑5公里，配合適當的重訓，體能進步很多
  情感: 中性
  正面分數: 0.00
  負面分數: 0.00
  信心度: 0.00

情感分類統計
正面文本數量: 2
負面文本數量: 1
中性文本數量: 1


In [17]:
# ==================== 2. 主題分類器 ====================

class TopicClassifier:
    def __init__(self):
        self.topic_keywords = {
            '科技': ['AI', '人工智慧', '電腦', '軟體', '程式', '演算法'],
            '運動': ['運動', '健身', '跑步', '游泳', '球類', '比賽'],
            '美食': ['吃', '食物', '餐廳', '美味', '料理', '烹飪'],
            '旅遊': ['旅行', '景點', '飯店', '機票', '觀光', '度假']
        }

    def classify(self, text):
        """返回最可能的主題"""
        topic_scores = {topic: 0 for topic in self.topic_keywords.keys()}

        # 計算每個主題的關鍵詞匹配數
        for topic, keywords in self.topic_keywords.items():
            for keyword in keywords:
                # 計算關鍵詞在文本中出現的次數
                count = text.count(keyword)
                topic_scores[topic] += count

        # 找出得分最高的主題
        max_score = max(topic_scores.values())

        if max_score == 0:
            return {
                'topic': '未分類',
                'confidence': 0,
                'scores': topic_scores
            }

        best_topic = max(topic_scores, key=topic_scores.get)

        return {
            'topic': best_topic,
            'confidence': max_score,
            'scores': topic_scores
        }


# ==================== 主題分類器測試 ====================

print("\n" + "=" * 80)
print("2. 主題分類器測試")
print("=" * 80)

topic_classifier = TopicClassifier()

for i, text in enumerate(test_texts, 1):
    print(f"\n測試文本 {i}: {text}")
    result = topic_classifier.classify(text)
    print(f"  主題: {result['topic']}")
    print(f"  信心度: {result['confidence']}")
    print(f"  各主題得分: {result['scores']}")


print("\n" + "=" * 80)
print("主題分類統計")
print("=" * 80)

topic_stats = {}
for text in test_texts:
    result = topic_classifier.classify(text)
    topic = result['topic']
    if topic not in topic_stats:
        topic_stats[topic] = 0
    topic_stats[topic] += 1

for topic, count in topic_stats.items():
    print(f"{topic}: {count} 篇")



2. 主題分類器測試

測試文本 1: 這家餐廳的牛肉麵真的太好吃了，湯頭濃郁，麵條Q彈，下次一定再來！
  主題: 美食
  信心度: 2
  各主題得分: {'科技': 0, '運動': 0, '美食': 2, '旅遊': 0}

測試文本 2: 最新的AI技術突破讓人驚艷，深度學習模型的表現越來越好
  主題: 科技
  信心度: 1
  各主題得分: {'科技': 1, '運動': 0, '美食': 0, '旅遊': 0}

測試文本 3: 這部電影劇情空洞，演技糟糕，完全是浪費時間
  主題: 未分類
  信心度: 0
  各主題得分: {'科技': 0, '運動': 0, '美食': 0, '旅遊': 0}

測試文本 4: 每天慢跑5公里，配合適當的重訓，體能進步很多
  主題: 未分類
  信心度: 0
  各主題得分: {'科技': 0, '運動': 0, '美食': 0, '旅遊': 0}

主題分類統計
美食: 1 篇
科技: 1 篇
未分類: 2 篇


In [18]:
import re

# A-3 統計式自動摘要

class StatisticalSummarizer:
    def __init__(self):
        # 定義停用詞 (過濾掉不具備實質意義的常用字)
        self.stop_words = set(['的', '了', '在', '是', '我', '有', '和',
                             '就', '不', '人', '都', '一', '一個', '上',
                             '也', '很', '到', '說', '要', '去', '你',
                             '這', '那', '會', '為', '著', '之', '與',
                             '及', '其', '或', '但', '而'])

    def sentence_score(self, sentence, word_freq, index=0, total_sentences=1):
        """
        計算句子重要性分數
        Args:
            sentence: 句子字串
            word_freq: 字詞頻率字典 (正規化後的)
            index: 句子在文章中的位置 (用於位置加權)
            total_sentences: 總句數
        """
        if not sentence:
            return 0

        score = 0
        # 簡單分詞：這裡以「字元」為單位，過濾停用詞
        words = [w for w in sentence if w not in self.stop_words]

        # 1. 關鍵詞頻率得分 (累加該句包含的詞之權重)
        for w in words:
            score += word_freq.get(w, 0)

        # 2. 位置加權 (Position Weighting)
        # 首句通常包含主旨，給予較高權重
        if index == 0:
            score *= 2.0
        # 尾句通常是總結，給予次高權重
        elif index == total_sentences - 1:
            score *= 1.5

        # 3. 長度懲罰 (可選)
        # 避免選到過短的句子
        if len(sentence) < 5:
            score *= 0.5

        return score

    def summarize(self, text, ratio=0.3):
        """
        摘要生成主程序
        Args:
            text: 原文
            ratio: 摘要比例 (例如 0.3 代表保留 30% 的句子)
        """
        # 1. 分句 (處理中文標點 。！？)
        # 使用正規表達式分割，並保留標點符號
        sentences = re.split(r'([。！？])', text)

        # 將標點符號接回句子後面
        clean_sentences = []
        for i in range(0, len(sentences)-1, 2):
            if sentences[i].strip():
                clean_sentences.append(sentences[i] + sentences[i+1])

        # 處理最後可能剩餘的部分
        if len(sentences) % 2 != 0 and sentences[-1].strip():
             clean_sentences.append(sentences[-1])

        if not clean_sentences:
            return "", []

        # 2. 計算全文字頻 (作為重要性依據)
        all_content = "".join(clean_sentences)
        all_words = [c for c in all_content if c not in self.stop_words and c.strip() and c not in '。！？，、：；「」『』']
        word_counts = Counter(all_words)

        # 正規化詞頻 (TF)
        total_words = len(all_words)
        word_freq_norm = {k: v/total_words for k, v in word_counts.items()}

        # 3. 計算每個句子的重要性分數
        sentence_scores = []
        for i, sent in enumerate(clean_sentences):
            score = self.sentence_score(sent, word_freq_norm, index=i, total_sentences=len(clean_sentences))
            sentence_scores.append((i, sent, score))

        # 4. 選擇最高分的句子
        num_sentences = max(1, int(len(clean_sentences) * ratio))

        # 依分數排序 (由高到低) 並選取前 N 句
        top_sentences = sorted(sentence_scores, key=lambda x: x[2], reverse=True)[:num_sentences]

        # 5. 按原文順序重新排列 (依 index 排序)
        top_sentences.sort(key=lambda x: x[0])

        # 組合結果
        summary = "".join([item[1] for item in top_sentences])

        return summary, sentence_scores

# ==================== 測試資料與執行 ====================

# 測試文章：關於人工智慧的短文
article = """人工智慧 (AI) 的發展正在深刻改變我們的生活方式。從早上起床時的智慧鬧鐘，到通勤時的路線規劃，再到工作中的各種輔助工具，AI無處不在。

在醫療領域，AI協助醫生進行疾病診斷，提高了診斷的準確率和效率。透過分析大量的醫療影像和病歷資料，AI能夠發現人眼容易忽略的細節，為患者提供更好的治療方案。

教育方面，AI個人化學習系統能夠根據每個學生的學習進度和特點，提供客製化的教學內容。這種因材施教的方式，讓學習變得更加高效和有趣。

然而，AI的快速發展也帶來了一些挑戰。首先是就業問題，許多傳統工作可能會被AI取代。其次是隱私和安全問題，AI系統需要大量數據來訓練，如何保護個人隱私成為重要議題。最後是倫理問題，AI的決策過程往往缺乏透明度，可能會產生偏見或歧視。

面對這些挑戰，我們需要在推動AI發展的同時，建立相應的法律法規和倫理準則。只有這樣，才能確保AI技術真正為人類福祉服務，創造一個更美好的未來。"""

print("=" * 80)
print("3. 統計式自動摘要測試")
print("=" * 80)

summarizer = StatisticalSummarizer()
summary, scores = summarizer.summarize(article, ratio=0.3)

print(f"\n原文長度: {len(article)} 字")
print(f"原文句數: {len(scores)} 句")
print(f"預計選取句數: {max(1, int(len(scores) * 0.3))} 句")
print("-" * 40)
print("各句子得分分析 (前5名):")

# 顯示分數最高的前5個句子
sorted_scores = sorted(scores, key=lambda x: x[2], reverse=True)
for i, (idx, sent, score) in enumerate(sorted_scores[:5], 1):
    print(f"Rank {i} (Index {idx}): {sent[:20]}... [Score: {score:.4f}]")

print("-" * 40)
print(f"生成摘要結果:\n")
print(summary)

3. 統計式自動摘要測試

原文長度: 407 字
原文句數: 12 句
預計選取句數: 3 句
----------------------------------------
各句子得分分析 (前5名):
Rank 1 (Index 0): 人工智慧 (AI) 的發展正在深刻改變我... [Score: 0.3907]
Rank 2 (Index 4): 

教育方面，AI個人化學習系統能夠根據... [Score: 0.3609]
Rank 3 (Index 11): 只有這樣，才能確保AI技術真正為人類福祉... [Score: 0.2980]
Rank 4 (Index 3): 透過分析大量的醫療影像和病歷資料，AI能... [Score: 0.2848]
Rank 5 (Index 8): 其次是隱私和安全問題，AI系統需要大量數... [Score: 0.2483]
----------------------------------------
生成摘要結果:

人工智慧 (AI) 的發展正在深刻改變我們的生活方式。

教育方面，AI個人化學習系統能夠根據每個學生的學習進度和特點，提供客製化的教學內容。只有這樣，才能確保AI技術真正為人類福祉服務，創造一個更美好的未來。
